In [11]:
import torch
import timm

# Step 1: Load the checkpoint dictionary (not just the raw weights)
checkpoint = torch.load('brain_tumor_deit_model.pth', map_location='cpu')

# Step 2: Create the model (ensure num_classes matches training)
model = timm.create_model('deit_small_patch16_224', pretrained=False, num_classes=4)

# Step 3: Load weights from the correct key
model.load_state_dict(checkpoint['model_state_dict'])

# Step 4: Set to eval mode
model.eval()


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # assuming RGB
])
test_dataset = datasets.ImageFolder('C:/Users/vikra/OneDrive/Desktop/NeuroVision-master/NeuroVision-master/Data/Testing', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [6]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')


Accuracy: 98.32%


In [9]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
# Evaluation
y_true = []
y_pred = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Metrics
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision (macro):", precision_score(y_true, y_pred, average='macro'))
print("Recall (macro):", recall_score(y_true, y_pred, average='macro'))
print("F1 Score (macro):", f1_score(y_true, y_pred, average='macro'))

# Detailed classification report
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=test_dataset.classes))

Accuracy: 0.9832189168573608
Precision (macro): 0.9820142714552788
Recall (macro): 0.9835257806826434
F1 Score (macro): 0.9826978877753517

Classification Report:
               precision    recall  f1-score   support

      glioma       0.96      0.99      0.97       300
  meningioma       0.98      0.97      0.98       306
     notumor       1.00      0.98      0.99       405
   pituitary       0.99      0.99      0.99       300

    accuracy                           0.98      1311
   macro avg       0.98      0.98      0.98      1311
weighted avg       0.98      0.98      0.98      1311



In [13]:
# Step 4: Collect predictions
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

# Step 5: Generate report
target_names = test_dataset.classes  # should be ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

      glioma       0.96      0.99      0.97       300
  meningioma       0.98      0.97      0.98       306
     notumor       1.00      0.98      0.99       405
   pituitary       0.99      0.99      0.99       300

    accuracy                           0.98      1311
   macro avg       0.98      0.98      0.98      1311
weighted avg       0.98      0.98      0.98      1311

